# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [8]:
*view_sentence_range

SyntaxError: can't use starred expression here (<ipython-input-8-7490453ae290>, line 1)

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [26]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    s = [[source_vocab_to_int.get(word, helper.CODES['<UNK>']) for word in sentence.split()] for sentence in source_text.split('\n')]
    t = [[target_vocab_to_int.get(word, helper.CODES['<UNK>']) for word in sentence.split()] + [target_vocab_to_int['<EOS>']] for sentence in target_text.split('\n')]
    
    return s, t

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    tsl = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_tsl = tf.reduce_max(tsl, name='max_target_len')
    ssl = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return input_data, targets, lr, keep_prob, tsl, max_tsl, ssl


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    enc_embed_in = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2))        
        return enc_cell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    drop = tf.contrib.rnn.DropoutWrapper(enc_cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)
    
    enc_out, enc_state = tf.nn.dynamic_rnn(drop, enc_embed_in, sequence_length=source_sequence_length, dtype=tf.float32)    
    
    return enc_out, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length, time_major=False)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    out = tf.contrib.seq2seq.dynamic_decode(decoder, impute_finished=True, maximum_iterations=max_summary_length)[0]
    
    return out

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [7]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    out = tf.contrib.seq2seq.dynamic_decode(decoder, impute_finished=True, maximum_iterations=max_target_sequence_length)[0]
    
    return out



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [8]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    def make_cell(rnn_size):
        cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2))        
        return cell
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    drop = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)
    output_layer = Dense(target_vocab_size, kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
        train_logits = decoding_layer_train(encoder_state, drop, dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, output_layer, keep_prob)
        
    with tf.variable_scope("decode", reuse=True):
        infer_logits = decoding_layer_infer(encoder_state, drop, dec_embeddings,
                                            target_vocab_to_int['<GO>'],
                                            target_vocab_to_int['<EOS>'],
                                            max_target_sequence_length, target_vocab_size, output_layer, batch_size, keep_prob)
        
        
    
    return train_logits, infer_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [9]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, source_sequence_length, source_vocab_size, 
                                  enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    train_dec_out, infer_dec_out = decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, 
                                                  rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                                                  batch_size, keep_prob, dec_embedding_size)
    
    return train_dec_out, infer_dec_out


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [10]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 120
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 1
display_step = 20

### Build the Graph
Build the graph using the neural network you implemented.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [44]:
save_path = 'checkpoints'

In [46]:
stop_val_acc = 0.95

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

training_done = False

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        if training_done: break
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))
                
                if(valid_acc > stop_val_acc):
                    saver = tf.train.Saver()
                    saver.save(sess, save_path)
                    print('Model Trained and Saved')
                    training_done = True
                    break

    # Save Model
#     saver = tf.train.Saver()
#     saver.save(sess, save_path)
#     print('Model Trained and Saved')

Epoch   0 Batch   20/1148 - Train Accuracy: 0.2525, Validation Accuracy: 0.3095, Loss: 4.6497
Epoch   0 Batch   40/1148 - Train Accuracy: 0.2675, Validation Accuracy: 0.3383, Loss: 3.7094
Epoch   0 Batch   60/1148 - Train Accuracy: 0.2671, Validation Accuracy: 0.3402, Loss: 3.4921
Epoch   0 Batch   80/1148 - Train Accuracy: 0.2808, Validation Accuracy: 0.3398, Loss: 3.2855
Epoch   0 Batch  100/1148 - Train Accuracy: 0.2800, Validation Accuracy: 0.3398, Loss: 3.2105
Epoch   0 Batch  120/1148 - Train Accuracy: 0.2812, Validation Accuracy: 0.3458, Loss: 3.1711
Epoch   0 Batch  140/1148 - Train Accuracy: 0.3013, Validation Accuracy: 0.3644, Loss: 3.1221
Epoch   0 Batch  160/1148 - Train Accuracy: 0.3500, Validation Accuracy: 0.3765, Loss: 2.9268
Epoch   0 Batch  180/1148 - Train Accuracy: 0.3452, Validation Accuracy: 0.4042, Loss: 2.9940
Epoch   0 Batch  200/1148 - Train Accuracy: 0.3408, Validation Accuracy: 0.4095, Loss: 2.9558
Epoch   0 Batch  220/1148 - Train Accuracy: 0.3567, Validati

Epoch   1 Batch  640/1148 - Train Accuracy: 0.4608, Validation Accuracy: 0.4955, Loss: 0.9872
Epoch   1 Batch  660/1148 - Train Accuracy: 0.4708, Validation Accuracy: 0.5072, Loss: 1.0077
Epoch   1 Batch  680/1148 - Train Accuracy: 0.4688, Validation Accuracy: 0.5114, Loss: 0.9827
Epoch   1 Batch  700/1148 - Train Accuracy: 0.4858, Validation Accuracy: 0.5117, Loss: 0.9864
Epoch   1 Batch  720/1148 - Train Accuracy: 0.4329, Validation Accuracy: 0.4909, Loss: 0.9779
Epoch   1 Batch  740/1148 - Train Accuracy: 0.4960, Validation Accuracy: 0.5197, Loss: 0.9458
Epoch   1 Batch  760/1148 - Train Accuracy: 0.5292, Validation Accuracy: 0.5042, Loss: 0.8771
Epoch   1 Batch  780/1148 - Train Accuracy: 0.4956, Validation Accuracy: 0.5034, Loss: 0.9355
Epoch   1 Batch  800/1148 - Train Accuracy: 0.4913, Validation Accuracy: 0.5057, Loss: 0.9416
Epoch   1 Batch  820/1148 - Train Accuracy: 0.4696, Validation Accuracy: 0.4852, Loss: 0.9494
Epoch   1 Batch  840/1148 - Train Accuracy: 0.4846, Validati

Epoch   3 Batch  120/1148 - Train Accuracy: 0.5929, Validation Accuracy: 0.6269, Loss: 0.6158
Epoch   3 Batch  140/1148 - Train Accuracy: 0.6254, Validation Accuracy: 0.6193, Loss: 0.6161
Epoch   3 Batch  160/1148 - Train Accuracy: 0.6377, Validation Accuracy: 0.6258, Loss: 0.5923
Epoch   3 Batch  180/1148 - Train Accuracy: 0.6105, Validation Accuracy: 0.6242, Loss: 0.6435
Epoch   3 Batch  200/1148 - Train Accuracy: 0.6258, Validation Accuracy: 0.6295, Loss: 0.6140
Epoch   3 Batch  220/1148 - Train Accuracy: 0.6092, Validation Accuracy: 0.6121, Loss: 0.5956
Epoch   3 Batch  240/1148 - Train Accuracy: 0.6271, Validation Accuracy: 0.6311, Loss: 0.5940
Epoch   3 Batch  260/1148 - Train Accuracy: 0.6500, Validation Accuracy: 0.6307, Loss: 0.5765
Epoch   3 Batch  280/1148 - Train Accuracy: 0.6017, Validation Accuracy: 0.6303, Loss: 0.5989
Epoch   3 Batch  300/1148 - Train Accuracy: 0.5942, Validation Accuracy: 0.6348, Loss: 0.6230
Epoch   3 Batch  320/1148 - Train Accuracy: 0.6184, Validati

Epoch   4 Batch  740/1148 - Train Accuracy: 0.6698, Validation Accuracy: 0.6330, Loss: 0.4100
Epoch   4 Batch  760/1148 - Train Accuracy: 0.7042, Validation Accuracy: 0.6492, Loss: 0.3743
Epoch   4 Batch  780/1148 - Train Accuracy: 0.6687, Validation Accuracy: 0.6337, Loss: 0.4114
Epoch   4 Batch  800/1148 - Train Accuracy: 0.6742, Validation Accuracy: 0.6386, Loss: 0.4343
Epoch   4 Batch  820/1148 - Train Accuracy: 0.6875, Validation Accuracy: 0.6731, Loss: 0.4128
Epoch   4 Batch  840/1148 - Train Accuracy: 0.6913, Validation Accuracy: 0.6561, Loss: 0.3828
Epoch   4 Batch  860/1148 - Train Accuracy: 0.6746, Validation Accuracy: 0.6689, Loss: 0.4185
Epoch   4 Batch  880/1148 - Train Accuracy: 0.6529, Validation Accuracy: 0.6705, Loss: 0.3998
Epoch   4 Batch  900/1148 - Train Accuracy: 0.6540, Validation Accuracy: 0.6553, Loss: 0.4051
Epoch   4 Batch  920/1148 - Train Accuracy: 0.6754, Validation Accuracy: 0.6663, Loss: 0.3920
Epoch   4 Batch  940/1148 - Train Accuracy: 0.6600, Validati

Epoch   6 Batch  220/1148 - Train Accuracy: 0.7850, Validation Accuracy: 0.7277, Loss: 0.2790
Epoch   6 Batch  240/1148 - Train Accuracy: 0.7333, Validation Accuracy: 0.7227, Loss: 0.2780
Epoch   6 Batch  260/1148 - Train Accuracy: 0.7432, Validation Accuracy: 0.7417, Loss: 0.2773
Epoch   6 Batch  280/1148 - Train Accuracy: 0.7050, Validation Accuracy: 0.7383, Loss: 0.2835
Epoch   6 Batch  300/1148 - Train Accuracy: 0.7054, Validation Accuracy: 0.7205, Loss: 0.3026
Epoch   6 Batch  320/1148 - Train Accuracy: 0.7671, Validation Accuracy: 0.7523, Loss: 0.2685
Epoch   6 Batch  340/1148 - Train Accuracy: 0.7671, Validation Accuracy: 0.7436, Loss: 0.2835
Epoch   6 Batch  360/1148 - Train Accuracy: 0.7171, Validation Accuracy: 0.7405, Loss: 0.3001
Epoch   6 Batch  380/1148 - Train Accuracy: 0.7733, Validation Accuracy: 0.7216, Loss: 0.2561
Epoch   6 Batch  400/1148 - Train Accuracy: 0.7720, Validation Accuracy: 0.7428, Loss: 0.2567
Epoch   6 Batch  420/1148 - Train Accuracy: 0.7313, Validati

...

Epoch  12 Batch  420/1148 - Train Accuracy: 0.9175, Validation Accuracy: 0.8890, Loss: 0.0669
Epoch  12 Batch  440/1148 - Train Accuracy: 0.9133, Validation Accuracy: 0.8970, Loss: 0.0604
Epoch  12 Batch  460/1148 - Train Accuracy: 0.9021, Validation Accuracy: 0.8799, Loss: 0.0598
Epoch  12 Batch  480/1148 - Train Accuracy: 0.9208, Validation Accuracy: 0.8689, Loss: 0.0722
Epoch  12 Batch  500/1148 - Train Accuracy: 0.8888, Validation Accuracy: 0.8629, Loss: 0.0767
Epoch  12 Batch  520/1148 - Train Accuracy: 0.9180, Validation Accuracy: 0.8898, Loss: 0.0672
Epoch  12 Batch  540/1148 - Train Accuracy: 0.8871, Validation Accuracy: 0.8864, Loss: 0.0879
Epoch  12 Batch  560/1148 - Train Accuracy: 0.8746, Validation Accuracy: 0.9030, Loss: 0.0829
Epoch  12 Batch  580/1148 - Train Accuracy: 0.9317, Validation Accuracy: 0.8917, Loss: 0.0613
Epoch  12 Batch  600/1148 - Train Accuracy: 0.8980, Validation Accuracy: 0.8826, Loss: 0.0752
Epoch  12 Batch  620/1148 - Train Accuracy: 0.9075, Validati

Epoch  13 Batch 1040/1148 - Train Accuracy: 0.9123, Validation Accuracy: 0.9117, Loss: 0.0631
Epoch  13 Batch 1060/1148 - Train Accuracy: 0.8921, Validation Accuracy: 0.9212, Loss: 0.0568
Epoch  13 Batch 1080/1148 - Train Accuracy: 0.9385, Validation Accuracy: 0.9193, Loss: 0.0429
Epoch  13 Batch 1100/1148 - Train Accuracy: 0.8912, Validation Accuracy: 0.9148, Loss: 0.0636
Epoch  13 Batch 1120/1148 - Train Accuracy: 0.9146, Validation Accuracy: 0.9072, Loss: 0.0607
Epoch  13 Batch 1140/1148 - Train Accuracy: 0.8948, Validation Accuracy: 0.8803, Loss: 0.0567
Epoch  14 Batch   20/1148 - Train Accuracy: 0.8879, Validation Accuracy: 0.9174, Loss: 0.0660
Epoch  14 Batch   40/1148 - Train Accuracy: 0.9563, Validation Accuracy: 0.8992, Loss: 0.0679
Epoch  14 Batch   60/1148 - Train Accuracy: 0.9163, Validation Accuracy: 0.8917, Loss: 0.0457
Epoch  14 Batch   80/1148 - Train Accuracy: 0.9083, Validation Accuracy: 0.9027, Loss: 0.0824
Epoch  14 Batch  100/1148 - Train Accuracy: 0.9342, Validati

Epoch  15 Batch  520/1148 - Train Accuracy: 0.9202, Validation Accuracy: 0.8985, Loss: 0.0471
Epoch  15 Batch  540/1148 - Train Accuracy: 0.9017, Validation Accuracy: 0.9189, Loss: 0.0672
Epoch  15 Batch  560/1148 - Train Accuracy: 0.8854, Validation Accuracy: 0.9261, Loss: 0.0626
Epoch  15 Batch  580/1148 - Train Accuracy: 0.9492, Validation Accuracy: 0.9193, Loss: 0.0378
Epoch  15 Batch  600/1148 - Train Accuracy: 0.9179, Validation Accuracy: 0.9129, Loss: 0.0564
Epoch  15 Batch  620/1148 - Train Accuracy: 0.9371, Validation Accuracy: 0.9000, Loss: 0.0407
Epoch  15 Batch  640/1148 - Train Accuracy: 0.9229, Validation Accuracy: 0.9106, Loss: 0.0463
Epoch  15 Batch  660/1148 - Train Accuracy: 0.9150, Validation Accuracy: 0.9258, Loss: 0.0421
Epoch  15 Batch  680/1148 - Train Accuracy: 0.9258, Validation Accuracy: 0.9193, Loss: 0.0443
Epoch  15 Batch  700/1148 - Train Accuracy: 0.9267, Validation Accuracy: 0.9239, Loss: 0.0488
Epoch  15 Batch  720/1148 - Train Accuracy: 0.9087, Validati

Epoch  16 Batch 1140/1148 - Train Accuracy: 0.9425, Validation Accuracy: 0.9246, Loss: 0.0410
Epoch  17 Batch   20/1148 - Train Accuracy: 0.8992, Validation Accuracy: 0.9163, Loss: 0.0465
Epoch  17 Batch   40/1148 - Train Accuracy: 0.9521, Validation Accuracy: 0.9330, Loss: 0.0543
Epoch  17 Batch   60/1148 - Train Accuracy: 0.9354, Validation Accuracy: 0.9205, Loss: 0.0326
Epoch  17 Batch   80/1148 - Train Accuracy: 0.9304, Validation Accuracy: 0.9205, Loss: 0.0637
Epoch  17 Batch  100/1148 - Train Accuracy: 0.9413, Validation Accuracy: 0.9330, Loss: 0.0361
Epoch  17 Batch  120/1148 - Train Accuracy: 0.8788, Validation Accuracy: 0.9231, Loss: 0.0531
Epoch  17 Batch  140/1148 - Train Accuracy: 0.9267, Validation Accuracy: 0.9356, Loss: 0.0456
Epoch  17 Batch  160/1148 - Train Accuracy: 0.9214, Validation Accuracy: 0.9307, Loss: 0.0442
Epoch  17 Batch  180/1148 - Train Accuracy: 0.9346, Validation Accuracy: 0.9341, Loss: 0.0533
Epoch  17 Batch  200/1148 - Train Accuracy: 0.9279, Validati

Epoch  18 Batch  620/1148 - Train Accuracy: 0.9425, Validation Accuracy: 0.9277, Loss: 0.0312
Epoch  18 Batch  640/1148 - Train Accuracy: 0.9387, Validation Accuracy: 0.9269, Loss: 0.0369
Epoch  18 Batch  660/1148 - Train Accuracy: 0.9458, Validation Accuracy: 0.9492, Loss: 0.0332
Epoch  18 Batch  680/1148 - Train Accuracy: 0.9467, Validation Accuracy: 0.9360, Loss: 0.0358
Epoch  18 Batch  700/1148 - Train Accuracy: 0.9217, Validation Accuracy: 0.9371, Loss: 0.0369
Epoch  18 Batch  720/1148 - Train Accuracy: 0.9353, Validation Accuracy: 0.9337, Loss: 0.0444
Epoch  18 Batch  740/1148 - Train Accuracy: 0.9361, Validation Accuracy: 0.9458, Loss: 0.0438
Epoch  18 Batch  760/1148 - Train Accuracy: 0.9496, Validation Accuracy: 0.9220, Loss: 0.0326
Epoch  18 Batch  780/1148 - Train Accuracy: 0.9198, Validation Accuracy: 0.9280, Loss: 0.0400
Epoch  18 Batch  800/1148 - Train Accuracy: 0.9300, Validation Accuracy: 0.9356, Loss: 0.0443
Epoch  18 Batch  820/1148 - Train Accuracy: 0.9354, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [48]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [50]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    seq = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]
    
    return seq


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [53]:
translate_sentence = 'the roof is on fire .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints
Input
  Word Ids:      [188, 2, 165, 2, 2, 58]
  English Words: ['the', '<UNK>', 'is', '<UNK>', '<UNK>', '.']

Prediction
  Word Ids:      [52, 267, 188, 181, 207, 1]
  French Words: l'éléphant est le printemps . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.